import data

In [2]:
import pickle
with open('../pickles/data_cleaned.pickle', 'rb') as handle:
    data_cleaned = pickle.load(handle)

In [3]:
nosleep2020 = data_cleaned['RS_2020_nosleep']

In [4]:
confessions2020 = data_cleaned['RS_2020_confessions']

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten
from tqdm.notebook import tqdm
import swifter
import pandas as pd
import numpy as np

In [8]:
def LSTMlastLayer(row):
    import spacy
    from sentence_transformers import SentenceTransformer, util
    import torch
    import numpy as np
    spacy.prefer_gpu()
    
    print(row)
    
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(row)
    
    embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    sents = [sent.text for sent in doc.sents]
    # encode gives shape (# of sents, 768,)
    data = embedder.encode(sents)
    # reshape the data to shape (# of sents, 768, 1)
    data = data.reshape(data.shape[0],data.shape[1],-1)
    
    LSTMmodel = Sequential([
        LSTM(768, input_shape = data.shape[1:], return_sequences = True),
        Dropout(0.1),
        LSTM(768),
        Dropout(0.1, name="lastLayer")
    ])

    intermediate_layer_model = tf.keras.Model(inputs=LSTMmodel.input,
                                     outputs=LSTMmodel.get_layer('lastLayer').output)
    intermediate_output = intermediate_layer_model.predict(data)
    
    return np.average(intermediate_output,axis=0)

In [9]:
nosleep2020['LSTMvec'] = nosleep2020.selftext.swifter.progress_bar(enable=True).apply(LSTMlastLayer)

KeyboardInterrupt: 

In [10]:
nosleep2020

,id,title,selftext,score,title_language,selftext_language
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1,en,en
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1,en,en
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1,de,en
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,en,en
6,gxrm8v,I'm tasked with killing nameless things out in...,"""Any sign of 'em yet?"" \n\nI continued staring...",1,en,en
...,...,...,...,...,...,...
21218,eihp0m,Hylophobia,*There is no cure for trauma. Once it enters t...,1,vi,en
21219,eihmg7,I adopted my late sisters orphaned child. This...,"I knew Persephone would need time to adjust, b...",1,en,en
21221,eihgtp,My first paranormal experience!!,"This isnt much, but this is surely the first u...",1,en,en
21223,eigzgj,I met the demon under my bed... Its not what I...,"Okay. for context, this story started about a ...",1,en,en
